In [36]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import plotly.express as px
import dash_table
import pandas as pd

#This code is from sample data
#df_transpose = pd.read_csv("detection_result_transpose.csv")
#this is simulated data
df_transpose = pd.read_csv("simulation_dash.csv")

grouped_company = df_transpose.groupby('Name').agg(
                                                    Name = ('Name',min),
                                                    Total_Non_Compliant_Cases = ('non-compliant_detected',sum),
                                                    Workers = ('Workers',sum)
                                                    )
df_company = pd.DataFrame(grouped_company)
df_company.eval("Non_Compliant_Worker_Ratio = Total_Non_Compliant_Cases/Workers", inplace=True)
print(df_company.head())
print(df_company.info())

"""

this code is from simulated data
df_simulation = pd.read_csv("simulation_dash.csv")
grouped_company_simulation = df_simulation.groupby('Name').agg(
                                                    Name = ('Name',min),
                                                    Total_Non_Compliant_Detected = ('non-compliant_detected',sum),
                                                    Workers = ('Workers',sum)
                                                    )

df_company_simulation = pd.DataFrame(grouped_company_simulation)
df_company_simulation.eval("Ratio = Total_Non_Compliant_Detected/Workers", inplace=True)
print(df_company_simulation.head())
print(df_company_simulation.info())
"""
app = dash.Dash()

                  Name  Total_Non_Compliant_Cases  Workers  \
Name                                                         
Company 0    Company 0                       96.0   2241.0   
Company 1    Company 1                      132.0   2653.0   
Company 10  Company 10                       37.0    496.0   
Company 11  Company 11                      132.0   1004.0   
Company 12  Company 12                      200.0   3865.0   

            Non_Compliant_Worker_Ratio  
Name                                    
Company 0                     0.042838  
Company 1                     0.049755  
Company 10                    0.074597  
Company 11                    0.131474  
Company 12                    0.051746  
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, Company 0 to Company 99
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Name                        100 non-null    o

In [37]:
#callback for graphs

@app.callback(
    [Output("line_graph", "figure")],
    [Input("company_name_dropdown", "value")],
)
def update_graph(value):
    dff_transpose = df_transpose[df_transpose['Name'] == value]
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=dff_transpose['Month'], y=dff_transpose['non-compliant_detected'], name='Non_Compliant_Detection',
                         line=dict(color='rgb(29,105,150)', width=4)))
    fig.add_trace(go.Scatter(x=dff_transpose['Month'], y=dff_transpose['claims'], name = 'Claims',
                         line=dict(color='rgb(218,165,27)', width=4)))
    # Edit the layout
    fig.update_layout(xaxis_title='Month',
                      yaxis_title='Total Number of Instances')
    return [fig]

#callback for tabs
@app.callback(Output('tabs-content-props', 'children'),
              [Input('tabs-styled-with-props', 'value')])
def render_content(tab):
    if tab == 'tab-1':
        return html.Div(
                        children=[
                            #html.H1(
                            #    children=["SunLife PPE Detection"],
                            #    style={'textAlign':'center'}
                            #),
                            html.H2(
                                children=["Non Compliance Cases Detected "],
                                style={'textAlign':'left'},
                            ),
                            html.H3(
                                children=["Risk Tolerence = 10% (0.1)"],
                                style={'textAlign':'left'},
                            ),
                            dcc.Graph(
                                id="bar_graph",
                                figure = px.bar(df_company, x="Name", y="Non_Compliant_Worker_Ratio", color='Non_Compliant_Worker_Ratio',color_continuous_scale=["green","red"],
                                                labels={'Name':'Company Name','Non_Compliant_Worker_Ratio':'Non-Compliance-Worker Ratio'}),
                            ),
                            html.H3(
                                children=["Select Company :"],
                                style={'textAlign':'left'},
                            ),
                            # https://dash.plotly.com/dash-core-components/dropdown
                            dcc.Dropdown(
                                id="company_name_dropdown",
                                options=[
                                    {'label':ticker,'value':ticker} for ticker in df_company['Name'].unique()
                                ],
                                value="Company 1",
                            ),
                             html.H2(
                                children=["Non-Compliant Detections Versus Claims"],
                                style={'textAlign':'left'},
                            ),
                            dcc.Graph(
                                id="line_graph",
                                figure={},    
                            ),
                            html.H2(
                                children=["Companies at Risk (Tolerance > 10%)"],
                                style={'textAlign':'left'},
                            ),
                            dash_table.DataTable(
                            id='table',
                            columns=[{"name": i, "id": i} for i in df_company.columns],
                            data=df_company[df_company['Non_Compliant_Worker_Ratio'] > 0.1].to_dict('records')),
                        ]
                    )
    elif tab == 'tab-2':
        return html.Div([
            html.H2('Non-Compliance Case Detection Live Updates!'),
            html.Iframe(src='/assets/TestsAug26.pdf',width = "90%", height="350px")
        ])

In [38]:

server = app.server

tabs_styles = {
    'height': '44px'
}
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': 'rgb(255,217,47)',
    'color': 'white',
    'padding': '6px'
}

app.layout =  html.Div([
     html.H1(children=["SunLife PPE Detection"],
             style={'textAlign':'center'}
            ),
    dcc.Tabs(id="tabs-styled-with-props", value='tab-1', children=[
        dcc.Tab(label='Trend', value='tab-1',style=tab_style, selected_style=tab_selected_style),
        dcc.Tab(label='Live', value='tab-2',style=tab_style, selected_style=tab_selected_style),
    ], style=tabs_styles
        #colors={
        #"border": "white",
        #"primary": "gold",
        #"background": "cornsilk"}
    ),
    html.Div(id='tabs-content-props')
])
    

### Step 4: build callbacks

### Step 5: run application

In [39]:
app.run_server(debug=False,port=8053)

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

Dash is running on http://127.0.0.1:8053/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server

 * Running on http://127.0.0.1:8053/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Sep/2020 23:21:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 23:21:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 23:21:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 23:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 23:21:16] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 23:21:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [01/Sep/2020 23:23:19] "POST /_dash-update-component HTTP/1.1" 200 -


In [93]:
%tb

SyntaxError: invalid syntax (<ipython-input-7-90d41e257c4d>, line 100)